In [1]:
MODEL_VERSION = "03"
NUM_TIMESTEPS = 28
RUN_ON_SAMPLE = True
SAMPLE_SIZE = 500
SCALING = False

DROPOUT = 0.3
MIN_LR = 1e-4
MAX_LR = 1e-2
STEP_SIZE = 2
BATCH_SIZE = 2*1024
PREDICT_BATCH_SIZE = 4*1024
NUM_EPOCHS = [2, 2, 2]

MC_DROPOUT = True
MC_SAMPLES = 100 if MC_DROPOUT else 1

In [2]:
import os, sys, datetime, pickle, gc
from time import time, ctime
from pprint import pprint
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv1D, Conv2D, LSTM, Embedding, Dense, concatenate, TimeDistributed
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Reshape, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import RepeatVector, Lambda
from tensorflow.keras.backend import repeat_elements
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

import tensorflow_addons as tfa
from tensorflow_addons.optimizers import TriangularCyclicalLearningRate
from tensorflow_addons.losses import PinballLoss

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
seeded_value = 88888
pd.set_option('display.max_colwidth', 50)
np.random.seed(seeded_value)
tf.random.set_seed(seeded_value)

In [5]:
# suppress scientific notation
pd.options.display.precision = 2
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [6]:
print([
    tf.__version__
])

['2.1.0']


In [7]:
start_time = time()
start = datetime.now()
print(ctime(start_time))

Sun Jun 28 04:48:28 2020


### Files
1. calendar.csv - Contains information about the dates on which the products are sold.
2. sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
3. sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.
4. sell_prices.csv - Contains information about the price of the products sold per store and date.
5. sales_train_evaluation.csv - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)

In [8]:
DATA_DIR = "../data/"
RESULTS_DIR = "../results/"
PICKLE_DIR = "../data/preprocessed/"

In [9]:
CARDINAL_VARS = ['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id', 'id']

In [10]:
TRAIN_INDICES = np.arange(1, 1885 + 1) # Could use 1156 i.e. only 2 years of data
VALID_INDICES = np.arange(1886, 1913 + 1)
PUBLIC_INDICES = np.arange(1914, 1941 + 1)
PRIVATE_INDICES = np.arange(1942, 1969 + 1)

In [11]:
print(min(TRAIN_INDICES), max(TRAIN_INDICES), len(TRAIN_INDICES))
print(min(VALID_INDICES), max(VALID_INDICES), len(VALID_INDICES))
print(min(PUBLIC_INDICES), max(PUBLIC_INDICES), len(PUBLIC_INDICES))
print(min(PRIVATE_INDICES), max(PRIVATE_INDICES), len(PRIVATE_INDICES))

1 1885 1885
1886 1913 28
1914 1941 28
1942 1969 28


Read about memory management in pandas [here](https://pythonspeed.com/articles/pandas-load-less-data/)

In [12]:
def diff_w_first(x:pd.Series, lag:int):
    x1 = x[:lag]
    x2  = x.diff(lag)[lag:]
    return pd.concat([x1, x2])

In [13]:
def add_features(df, grouping_col, col, roll_sizes, lag_sizes):
    
    # per series
    df[col+"_mean_s"] = df.groupby(grouping_col)[col].transform("mean")
    df[col+"_std_s"] = df.groupby(grouping_col)[col].transform("std")
    df[col+"_min_s"] = df.groupby(grouping_col)[col].transform("min")
    df[col+"_max_s"] = df.groupby(grouping_col)[col].transform("max")
    df[col+"_scaled_s"] = (df[col] - df[col+"_mean_s"]) / df[col+"_std_s"]
    
    # rolling window per series
    for i in roll_sizes:
        j = str(i)
        df[col+"_mean_rs_"+j] = df.groupby(grouping_col)[col].rolling(window=i, min_periods=1).mean().reset_index(drop=True)
        df[col+"_min_rs_"+j] = df.groupby(grouping_col)[col].rolling(window=i, min_periods=1).min().reset_index(drop=True)
        df[col+"_max_rs_"+j] = df.groupby(grouping_col)[col].rolling(window=i, min_periods=1).max().reset_index(drop=True)
        df[col+"_std_rs_"+j] = df.groupby(grouping_col)[col].rolling(window=i, min_periods=1).std().reset_index(drop=True)
        df[col+"_std_rs_"+j] = df[col+"_std_rs_"+j].fillna(1e-3)
    
    # various lags
    for i in lag_sizes:
        j = str(i)
        df[col+"_lag_"+j] = df.groupby(grouping_col)[col].apply(lambda x: diff_w_first(x,i)).reset_index(drop=False)[col]
    
    return df

In [14]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [15]:
def add_id_after_aggregations(df):
    subs = pd.read_csv('../data/sample_submission_uncertainty.csv')
    subs['id_interim'] = subs['id'].str.split("_").apply(lambda x: "_".join(x[:-2]).replace("_X",""))
    subs['id_trimmed'] = subs.id.str.split("_").apply(lambda x: "_".join(x[:-2]))

    df['id_interim'] = ''
    df['id_interim'].loc[(~df['item_id'].isna()) & (~df['store_id'].isna())] = df[['item_id', 'store_id']].fillna("").apply(lambda x: "_".join([i for i in x if i!=""]), axis=1)
    df['id_interim'].loc[(df['item_id'].isna()) | (df['store_id'].isna())] = df[['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']].fillna("").apply(lambda x: "_".join([i for i in x if i!=""]), axis=1)
    df['id_interim'].loc[df['id_interim']==''] = 'Total'

    df2 = df.merge(subs.loc[(subs.id.str.endswith('_evaluation')) & (subs.id.str.contains('0.005'))][['id_trimmed','id_interim']],
                   how='left', on='id_interim')

    df2.rename(columns={'id_trimmed':'id'}, inplace=True)
    df2.drop('id_interim', axis=1, inplace=True)
    return df2[['id','dept_id','cat_id','store_id','state_id','item_id'] + ['d_'+str(i) for i in range(1,1942)]]

In [16]:
def read_data(infer_mode:bool=False):
    print('Reading files...')
    
    calendar = pd.read_csv('../data/calendar.csv').fillna("None")
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    
    sell_prices = pd.read_csv('../data/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    
    sales = pd.read_csv('../data/sales_train_evaluation_with_aggregations.csv')
    sales = add_id_after_aggregations(df=sales)
    print('Sales train validation has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))
    sales[['d_'+str(i) for i in PRIVATE_INDICES]] = pd.DataFrame(np.zeros(shape=(sales.shape[0], len(PRIVATE_INDICES))))
    print('Sales train validation has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))
    
    submission = pd.read_csv('../data/sample_submission.csv')
    
    pprint({
        "calendar.shape" : calendar.shape,
        "sell_prices.shape" : sell_prices.shape,
        "sales.shape" : sales.shape,
        "submission.shape" : submission.shape
    })
    
    return calendar, sell_prices, sales, submission

In [17]:
def add_dateparts(calendar, datecolname):
    calendar["Date"] = pd.to_datetime(calendar[datecolname], format = ("%Y-%m-%d"))
    calendar["Year"] = calendar["Date"].dt.year.astype('int16')
    calendar["Quarter"] = calendar["Date"].dt.quarter
    calendar["Month"] = calendar["Date"].dt.month
    calendar["Week"] = calendar["Date"].dt.week
    calendar["Day"] = calendar["Date"].dt.day
    calendar["DOW"] = calendar["Date"].dt.dayofweek
    calendar = reduce_mem_usage(calendar)
    return calendar

In [18]:
def melt_join_fill(sales, calendar, sell_prices):
    
    print("[INFO] ", "Starting  -- COMPLETE", ctime(time()))
    if RUN_ON_SAMPLE:
        sales = pd.melt(sales.sample(SAMPLE_SIZE),
                        id_vars=CARDINAL_VARS,
                        value_vars=Y_VARS,
                        var_name="day_id",
                        value_name='demand')
    else:
        sales = pd.melt(sales,
                        id_vars=CARDINAL_VARS,
                        value_vars=Y_VARS,
                        var_name="day_id",
                        value_name='demand')
    print("[INFO] ", "Melting  -- COMPLETE", ctime(time()))

    gc.collect()
    
    sales = pd.merge(sales, calendar, how="left", left_on="day_id", right_on="d")
    print("[INFO] ", "Merging1 -- COMPLETE", ctime(time()))
    
    gc.collect()
    
    sales = pd.merge(sales, sell_prices, how="left", on=["wm_yr_wk", "store_id", "item_id"])
    print("[INFO] ", "Merging2 -- COMPLETE", ctime(time()))
    
    gc.collect()
    
    # if sell_price is NA
    sales = sales.sort_values(by=CARDINAL_VARS+["date"])
    print("[INFO] ", "Sorting -- COMPLETE", ctime(time()))
    
    # remove till first non-zero demand per series
    before = sales.shape
    sales['cumul_demand'] = sales.groupby(['id'])['demand'].cumsum()
    sales = sales.loc[sales['cumul_demand']>0].copy().reset_index(drop=True)
    after = sales.shape
    print("[INFO] ", "Removal -- COMPLETE", ctime(time()))
    print("[INFO] ", "Removal", before, "--->", after)
    
    gc.collect()
    
    sales["sell_price_available"] = np.where(sales.sell_price.isna(), "N", "Y")
    sales[["sell_price"]] = sales.groupby(["item_id"])[["sell_price"]].ffill()
    sales[["sell_price"]] = sales.groupby(["item_id"])[["sell_price"]].bfill()
    sales["sell_price"] = sales["sell_price"].fillna(3.5) # median price
    sales = sales.drop(["d", "wday", 'date', 'wm_yr_wk', 'weekday', 'month', 'year'], axis=1)
    
    sales['day_id'] = sales['day_id'].astype(str).apply(lambda x: x[2:]).astype(np.int16)
    sales[['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']] = sales[['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']].fillna("All")
    print("[INFO] ", "Imputing -- COMPLETE", ctime(time()))
    
    gc.collect()
    
    sales['weights'] = sales["sell_price"] * sales["demand"]
    sales['rolling_weights'] = sales.groupby(CARDINAL_VARS)['weights'].rolling(window=NUM_TIMESTEPS,
                                                                               min_periods=1).sum().reset_index(drop=True)
    
    sales['weights'] = sales['weights'].apply(lambda x: np.max((1.0, x)))
    sales['rolling_weights'] = sales['rolling_weights'].apply(lambda x: np.max((1.0, x)))
    
    sales['weights'] = sales['weights'].fillna(sales['weights'].max())
    sales['rolling_weights'] = sales['rolling_weights'].fillna(sales['rolling_weights'].max())
    
    print("[INFO] ", "Weighting -- COMPLETE", ctime(time()))    
    
    gc.collect()
    
    print("[INFO] ", "Final dataset contains", sales.shape)
    
    return sales.reset_index(drop=True)

In [19]:
if not os.path.exists(PICKLE_DIR+"merged_df.pickle"):
    
    calendar, sell_prices, sales, submission = read_data()
    
    calendar = add_dateparts(calendar, "date")
    
    Y_VARS = sales.columns[sales.columns.str.startswith("d_")]

    data = melt_join_fill(sales, calendar, sell_prices)
    
    del sales, calendar, sell_prices
    gc.collect()

    data["demand_lag"] = data.groupby('id')['demand'].apply(lambda x: diff_w_first(x,60)).reset_index(drop=False)['demand']
    data = add_features(df=data, grouping_col="id", col="demand_lag", roll_sizes=[7, 30, 90, 365], lag_sizes=[7, 30, 90, 365])

    print("Saving all data to ---> ", PICKLE_DIR)
    with open(os.path.join(PICKLE_DIR, "merged_df.pickle"),"wb") as f:
        pickle.dump((data), f)
else:
    print("Pickle exists hence loading from pickle file ---> ", PICKLE_DIR)
    with open(os.path.join(PICKLE_DIR+"merged_df.pickle"), "rb") as f:
        data = pickle.load(f)

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns


c:\users\deepak\miniconda3\envs\dev\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (0,1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\deepak\miniconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Sales train validation has 42840 rows and 1947 columns
Sales train validation has 42840 rows and 1975 columns
{'calendar.shape': (1969, 14),
 'sales.shape': (42840, 1975),
 'sell_prices.shape': (6841121, 4),
 'submission.shape': (60980, 29)}
Mem. usage decreased to  0.15 Mb (30.4% reduction)
[INFO]  Starting  -- COMPLETE Sun Jun 28 04:48:49 2020
[INFO]  Melting  -- COMPLETE Sun Jun 28 04:48:50 2020
[INFO]  Merging1 -- COMPLETE Sun Jun 28 04:48:50 2020
[INFO]  Merging2 -- COMPLETE Sun Jun 28 04:48:54 2020
[INFO]  Sorting -- COMPLETE Sun Jun 28 04:48:57 2020
[INFO]  Removal -- COMPLETE Sun Jun 28 04:48:57 2020
[INFO]  Removal (984500, 30) ---> (794276, 31)
[INFO]  Imputing -- COMPLETE Sun Jun 28 04:48:59 2020
[INFO]  Weighting -- COMPLETE Sun Jun 28 04:49:14 2020
[INFO]  Final dataset contains (794276, 27)
Saving all data to --->  ../data/preprocessed/


In [20]:
print(data.columns)

Index(['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id', 'id', 'day_id',
       'demand', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'Date', 'Year',
       'Quarter', 'Month', 'Week', 'Day', 'DOW', 'sell_price', 'cumul_demand',
       'sell_price_available', 'weights', 'rolling_weights', 'demand_lag',
       'demand_lag_mean_s', 'demand_lag_std_s', 'demand_lag_min_s',
       'demand_lag_max_s', 'demand_lag_scaled_s', 'demand_lag_mean_rs_7',
       'demand_lag_min_rs_7', 'demand_lag_max_rs_7', 'demand_lag_std_rs_7',
       'demand_lag_mean_rs_30', 'demand_lag_min_rs_30', 'demand_lag_max_rs_30',
       'demand_lag_std_rs_30', 'demand_lag_mean_rs_90', 'demand_lag_min_rs_90',
       'demand_lag_max_rs_90', 'demand_lag_std_rs_90',
       'demand_lag_mean_rs_365', 'demand_lag_min_rs_365',
       'demand_lag_max_rs_365', 'demand_lag_std_rs_365', 'demand_lag_lag_7',
       'demand_lag_lag_30', 'demand_lag_lag_90', 'demand_lag_l

In [21]:
1913-28, 1913, 1913 + 28, 1913 + 28 + 28

(1885, 1913, 1941, 1969)

In [22]:
data.Date.min(), data.Date.max()#, data.Date.max() + 28, data.Date.max() + 28 + 28

(Timestamp('2011-01-29 00:00:00'), Timestamp('2016-06-19 00:00:00'))

In [23]:
print(data.shape); data.head()

(794276, 53)


,state_id,store_id,cat_id,dept_id,item_id,id,day_id,demand,event_name_1,event_type_1,...,demand_lag_max_rs_90,demand_lag_std_rs_90,demand_lag_mean_rs_365,demand_lag_min_rs_365,demand_lag_max_rs_365,demand_lag_std_rs_365,demand_lag_lag_7,demand_lag_lag_30,demand_lag_lag_90,demand_lag_lag_365
0,CA,All,All,All,FOODS_1_001,CA_FOODS_1_001,1,6.00,None,None,...,538.00,0.00,538.00,538.00,538.00,0.00,6.00,6.00,6.00,6.00
1,CA,All,All,All,FOODS_1_001,CA_FOODS_1_001,2,3.00,None,None,...,538.00,99.70,467.50,397.00,538.00,99.70,3.00,3.00,3.00,3.00
2,CA,All,All,All,FOODS_1_001,CA_FOODS_1_001,3,2.00,None,None,...,538.00,90.94,434.33,368.00,538.00,90.94,2.00,2.00,2.00,2.00
3,CA,All,All,All,FOODS_1_001,CA_FOODS_1_001,4,3.00,None,None,...,538.00,85.39,413.25,350.00,538.00,85.39,3.00,3.00,3.00,3.00
4,CA,All,All,All,FOODS_1_001,CA_FOODS_1_001,5,7.00,None,None,...,538.00,90.65,389.80,296.00,538.00,90.65,7.00,7.00,7.00,7.00


# Preprocessing

In [24]:
CONT_VARS = ['snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
             'demand_lag', 'demand_lag_mean_s', 'demand_lag_std_s', 'demand_lag_min_s',
             'demand_lag_max_s', 'demand_lag_scaled_s', 'demand_lag_mean_rs_7',
             'demand_lag_min_rs_7', 'demand_lag_max_rs_7', 'demand_lag_std_rs_7',
             'demand_lag_mean_rs_30', 'demand_lag_min_rs_30', 'demand_lag_max_rs_30',
             'demand_lag_std_rs_30', 'demand_lag_mean_rs_90', 'demand_lag_min_rs_90',
             'demand_lag_max_rs_90', 'demand_lag_std_rs_90', 'demand_lag_mean_rs_365',
             'demand_lag_min_rs_365', 'demand_lag_max_rs_365', 'demand_lag_std_rs_365',
             'demand_lag_lag_7', 'demand_lag_lag_30', 'demand_lag_lag_90', 'demand_lag_lag_365']
CAT_VARS = ['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id', 'event_name_1',
            'event_type_1', 'event_name_2', 'event_type_2', 'Year', 'Quarter',
            'Month', 'Week', 'Day', 'DOW', 'sell_price_available']
DEP_VAR = ['demand']
WEIGHT_VAR = ['weights', 'rolling_weights']

### Categorical Data

In [23]:
Category_Mapping = {}
for c in CAT_VARS:
    # convert columns to categories
    data[c+"_cat"] = data[c].astype("category")
    
    # save the mapping for later use
    Category_Mapping.update({c+"_cat" : dict(enumerate(data[c+"_cat"].cat.categories))})
    
    # Copy categories as integer codes
    data[c+"_cat"] = data[c+"_cat"].cat.codes

In [24]:
Unique_Dict ={}
for c in CAT_VARS:
    col = c+"_cat"
    Unique_Dict.update({col:{'min':data[col].min(), 
                             'max':data[col].max(), 
                             'nuniq':data[col].nunique(), 
                             'emb_sz':max(min(int(data[col].nunique() / 2), 10), 2)}})
pprint(Unique_Dict)

{'DOW_cat': {'emb_sz': 3, 'max': 6, 'min': 0, 'nuniq': 7},
 'Day_cat': {'emb_sz': 10, 'max': 30, 'min': 0, 'nuniq': 31},
 'Month_cat': {'emb_sz': 6, 'max': 11, 'min': 0, 'nuniq': 12},
 'Quarter_cat': {'emb_sz': 2, 'max': 3, 'min': 0, 'nuniq': 4},
 'Week_cat': {'emb_sz': 10, 'max': 52, 'min': 0, 'nuniq': 53},
 'Year_cat': {'emb_sz': 3, 'max': 5, 'min': 0, 'nuniq': 6},
 'cat_id_cat': {'emb_sz': 2, 'max': 1, 'min': 0, 'nuniq': 2},
 'dept_id_cat': {'emb_sz': 2, 'max': 0, 'min': 0, 'nuniq': 1},
 'event_name_1_cat': {'emb_sz': 10, 'max': 30, 'min': 0, 'nuniq': 31},
 'event_name_2_cat': {'emb_sz': 2, 'max': 4, 'min': 0, 'nuniq': 5},
 'event_type_1_cat': {'emb_sz': 2, 'max': 4, 'min': 0, 'nuniq': 5},
 'event_type_2_cat': {'emb_sz': 2, 'max': 2, 'min': 0, 'nuniq': 3},
 'item_id_cat': {'emb_sz': 10, 'max': 461, 'min': 0, 'nuniq': 462},
 'sell_price_available_cat': {'emb_sz': 2, 'max': 1, 'min': 0, 'nuniq': 2},
 'state_id_cat': {'emb_sz': 2, 'max': 3, 'min': 0, 'nuniq': 4},
 'store_id_cat': {'emb

In [25]:
NEW_CAT_VARS = [col+"_cat" for col in CAT_VARS]

### Numerical Data

In [26]:
if SCALING:
    MMS = MinMaxScaler()
    data[CONT_VARS] = MMS.fit_transform(data[CONT_VARS])

    MMS_Y = MinMaxScaler(feature_range=(0, 0.8))
    data[DEP_VAR] = MMS_Y.fit_transform(data[DEP_VAR])

# Cross Validation

In [28]:
data['set'] = np.where(data.day_id.isin(TRAIN_INDICES), "Train",
                       np.where(data.day_id.isin(VALID_INDICES), "Valid",
                                np.where(data.day_id.isin(PUBLIC_INDICES), "Public", "Private")))

In [28]:
data.groupby('set').agg({
    'id':'count',
    'day_id':[np.min, np.max]
})

id day_id      
          count   amin  amax
set                         
Private   14000   1942  1969
Public    14000   1914  1941
Train    752276      1  1885
Valid     14000   1886  1913

In [29]:
train_data = data.loc[data.day_id.isin(TRAIN_INDICES)]
valid_data = data.loc[data.day_id.isin(VALID_INDICES)]
public_data = data.loc[data.day_id.isin(PUBLIC_INDICES)]
private_data = data.loc[data.day_id.isin(PRIVATE_INDICES)]

print(train_data.shape, valid_data.shape, public_data.shape, private_data.shape)

(752276, 44) (14000, 44) (14000, 44) (14000, 44)


In [30]:
data.shape[0] == train_data.shape[0] + valid_data.shape[0] + private_data.shape[0] + private_data.shape[0]

True

In [31]:
model_time = time()
model_t = datetime.now()
print(ctime(model_time))

Sun Jun 28 03:18:28 2020


# Model Specification

In [51]:
def build_model():
    layers = []
    inputs = []
    for i,col in enumerate(CAT_VARS):
        input_ = Input(shape=1, name=col+"_cat")
        embedding =  Embedding(Unique_Dict[col+"_cat"]['nuniq'],
                               Unique_Dict[col+"_cat"]['emb_sz'],
                               name='emb_'+col)(input_)
        vec = Flatten()(embedding)
        layers.append(vec)
        inputs.append(input_)
    
    for i, col in enumerate(CONT_VARS):
        input_ = Input(shape=1, name=col)
        layers.append(input_)
        inputs.append(input_)
    
    concat_layer = concatenate(layers)
    x = Dense(2048)(concat_layer)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)

    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)

    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)
    
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)
    
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)
    
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)
    
    x = Dense(32)(x)
    x = Dropout(DROPOUT)(x, training=MC_DROPOUT)
    x = Activation('relu')(x)
    
    if SCALING:
        demand = Dense(1, activation='sigmoid', name='demand')(x)
    else:
        demand = Dense(1, activation='relu', name='demand')(x)
    
    model = Model(inputs, demand)
    
    return model

In [52]:
model = build_model()

In [53]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
state_id_cat (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
store_id_cat (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
cat_id_cat (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
dept_id_cat (InputLayer)        [(None, 1)]          0                                            
____________________________________________________________________________________________

In [54]:
tclr = TriangularCyclicalLearningRate(
    initial_learning_rate=MIN_LR,
    maximal_learning_rate=MAX_LR,
    step_size=STEP_SIZE * len(train_data)
)

In [55]:
mcp = ModelCheckpoint(filepath=RESULTS_DIR+"BestCheckpoint_"+MODEL_VERSION+".h5", monitor='val_loss',
                      verbose=0, save_best_only=True, save_weights_only=False, save_freq='epoch')
csvl = CSVLogger(filename=RESULTS_DIR+"LossLogs_"+MODEL_VERSION+".csv",
                 separator=",", append=True)

In [56]:
PBL = PinballLoss(tau=[0.005,0.025,0.165,0.25,0.5,0.75,0.835,0.975,0.995])

In [57]:
adam = Adam(learning_rate=MAX_LR)
model.compile(loss= PBL if MC_DROPOUT else 'mse',
              optimizer=adam)

In [58]:
history = model.fit(x=train_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                    y=train_data[DEP_VAR].to_dict(orient='series'),
                    validation_data=(valid_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                     valid_data[DEP_VAR].to_dict(orient='series'),
                                     valid_data[WEIGHT_VAR].to_dict(orient='series')['rolling_weights']
                                    ),
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS[0],
                    shuffle=True,
                    verbose=1,
                    sample_weight=train_data[WEIGHT_VAR].to_dict(orient='series')['rolling_weights'],
                    callbacks=[mcp, csvl])

  ...
    to  
  ['...']
Train on 752276 samples, validate on 14000 samples
Epoch 1/2
752276/752276 [==============================] - 15s 20us/sample - loss: 10.2259 - val_loss: nan
Epoch 2/2
 12288/752276 [..............................] - ETA: 10s - loss: 9.0363

c:\users\deepak\miniconda3\envs\dev\lib\site-packages\tensorflow_core\python\keras\callbacks.py:1020: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):


210944/752276 [=======>......................] - ETA: 8s - loss: 9.9005WARNING:tensorflow:Can save best model only with val_loss available, skipping.


KeyError: 'val_loss'

In [ ]:
metric_names = ['loss']

for i, j in zip(metric_names, ['val_'+i for i in metric_names]):
    plt.plot(history.history[i])
    plt.plot(history.history[j])
    plt.title('Model '+i)
    plt.ylabel(i.upper())
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

In [ ]:
model = tf.keras.models.load_model(filepath=RESULTS_DIR+"BestCheckpoint_"+MODEL_VERSION+".h5", compile=False)

In [ ]:
adam = Adam(learning_rate=tclr)
model.compile(loss=PBL if MC_DROPOUT else 'mse',
              optimizer=adam)

In [ ]:
history = model.fit(x=train_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                    y=train_data[DEP_VAR].to_dict(orient='series'),
                    validation_data=(valid_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                     valid_data[DEP_VAR].to_dict(orient='series'),
                                     valid_data[WEIGHT_VAR].to_dict(orient='series')['rolling_weights']
                                    ),
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS[1],
                    shuffle=True,
                    verbose=1, 
                    sample_weight=train_data[WEIGHT_VAR].to_dict(orient='series')['rolling_weights'],
                    callbacks=[mcp, csvl])

In [ ]:
metric_names = ['loss']

for i, j in zip(metric_names, ['val_'+i for i in metric_names]):
    plt.plot(history.history[i])
    plt.plot(history.history[j])
    plt.title('Model ' + i)
    plt.ylabel(i.upper())
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

In [ ]:
model = tf.keras.models.load_model(filepath=RESULTS_DIR+"BestCheckpoint_"+MODEL_VERSION+".h5", compile=False)

In [ ]:
adam = Adam(learning_rate=MIN_LR*0.1)
model.compile(loss=PBL if MC_DROPOUT else 'mse',
              optimizer=adam)

In [ ]:
history = model.fit(x=train_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                    y=train_data[DEP_VAR].to_dict(orient='series'),
                    validation_data=(valid_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                     valid_data[DEP_VAR].to_dict(orient='series'),
                                     valid_data[WEIGHT_VAR].to_dict(orient='series')['rolling_weights']
                                    ),
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS[2],
                    shuffle=True,
                    verbose=1, 
                    sample_weight=train_data[WEIGHT_VAR].to_dict(orient='series')['rolling_weights'],
                    callbacks=[mcp, csvl])

In [ ]:
metric_names = ['loss']

for i, j in zip(metric_names, ['val_'+i for i in metric_names]):
    plt.plot(history.history[i])
    plt.plot(history.history[j])
    plt.title('Model '+i)
    plt.ylabel(i.upper())
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

In [ ]:
model.save(filepath=RESULTS_DIR+"FinalCheckpoint_"+MODEL_VERSION+".h5", overwrite=True)

# Validation

In [ ]:
if os.path.exists(RESULTS_DIR+"BestCheckpoint_"+MODEL_VERSION+".h5"):
    model = tf.keras.models.load_model(filepath=RESULTS_DIR+"BestCheckpoint_"+MODEL_VERSION+".h5", compile=False)

In [ ]:
for i in range(MC_SAMPLES):
    if i == 0:
        train_preds = model.predict(train_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                    batch_size=PREDICT_BATCH_SIZE, verbose=0)

        valid_preds = model.predict(valid_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                    batch_size=PREDICT_BATCH_SIZE, verbose=0)

        public_preds = model.predict(public_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                     batch_size=PREDICT_BATCH_SIZE, verbose=0)

        private_preds = model.predict(private_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                      batch_size=PREDICT_BATCH_SIZE, verbose=0)
        
    else:
        train_preds = np.concatenate((train_preds, 
                                      model.predict(train_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                                    batch_size=PREDICT_BATCH_SIZE, verbose=0)), axis=1)

        valid_preds = np.concatenate((valid_preds, 
                                      model.predict(valid_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                                    batch_size=PREDICT_BATCH_SIZE, verbose=0)), axis=1)

        public_preds = np.concatenate((public_preds, 
                                       model.predict(public_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                                     batch_size=PREDICT_BATCH_SIZE, verbose=0)), axis=1)

        private_preds = np.concatenate((private_preds, 
                                        model.predict(private_data[NEW_CAT_VARS + CONT_VARS].to_dict(orient='series'),
                                                      batch_size=PREDICT_BATCH_SIZE, verbose=0)), axis=1)

print(train_preds.shape, valid_preds.shape, public_preds.shape, private_preds.shape)

## Plotting

In [ ]:
mu, sigma, median = train_preds.mean(axis=1), train_preds.std(axis=1), np.median(train_preds, axis=1)

In [ ]:
train_preds[train_preds<0].sum(), valid_preds[valid_preds<0].sum(), public_preds[public_preds<0].sum(), private_preds[private_preds<0].sum(), 

In [ ]:
def plot_normals(preds, ex_index):
    num_bins = 100
    mu_, sigma_ = mu[ex_index], sigma[ex_index]

    n, bins, patches = plt.hist(preds[ex_index], num_bins,  
                                density = 1,  
                                color ='green', 
                                alpha = 0.7) 

    y = ((1 / (np.sqrt(2 * np.pi) * sigma_)) * np.exp(-0.5 * (1 / sigma_ * (bins - mu_))**2))

    plt.plot(bins, y, '--', color ='black') 

    plt.xlabel('X-Axis')
    plt.ylabel('Y-Axis')

    plt.title('Prediction Range Plot:'+str(ex_index), fontweight ="bold") 

    plt.show() 

In [ ]:
plot_normals(train_preds, np.random.randint(low=0, high=train_preds.shape[0], size=1)[0])

In [ ]:
plot_normals(train_preds, mu.argmax())

In [ ]:
plot_normals(train_preds, sigma.argmax())

In [ ]:
plot_normals(train_preds, median.argmax())

## Prediction Intervals

In [ ]:
def add_pred_stats(preds, data):
    data['pred_mean'], data['pred_std'], data['p_0.500'] = preds.mean(axis=1), preds.std(axis=1), np.median(preds, axis=1)
    return data

In [ ]:
Qs = np.array([0.5, 0.67, 0.95, 0.99])
min_cols = ['p_'+str(min_col) for min_col in np.round((1-Qs)/2,3)]
max_cols = ['p_'+str(max_col) for max_col in np.round((1+Qs)/2,3)]

In [ ]:
from scipy.stats import norm

def create_intervals(preds, data):

    Qs = np.array([0.5, 0.67, 0.95, 0.99])
    min_cols = ['p_0.250', 'p_0.165', 'p_0.025', 'p_0.005'] #['p_'+str(min_col) for min_col in np.round((1-Qs)/2,3)]
    max_cols = ['p_0.750', 'p_0.835', 'p_0.975', 'p_0.995'] #['p_'+str(max_col) for max_col in np.round((1+Qs)/2,3)]
    
    mu, sigma, median = preds.mean(axis=1), preds.std(axis=1), np.median(preds, axis=1)
    data['p_mean'] = mu
    for i, min_col, max_col in zip(Qs, min_cols, max_cols):
        min_values, max_values = norm.interval(alpha=i, loc=mu, scale=sigma)
        
        sigma_flag = sigma==0
        min_values[sigma_flag], max_values[sigma_flag] = 0, 0
        print(i, "Altered",sum(sigma_flag),"rows due to zero sigma")
        
        min_neg_flag = min_values<0; max_neg_flag = max_values<0; 
        min_values[min_neg_flag], max_values[max_neg_flag] = 0, 0
        print(i, "Altered",sum(min_neg_flag), sum(max_neg_flag),"rows due to less than zero intervals")
        
        data[min_col], data[max_col] = min_values, max_values
    
    data[min_cols + max_cols] = data[min_cols + max_cols].fillna(0)
    
    return data

In [ ]:
train_data = add_pred_stats(preds=train_preds, data=train_data)
train_data = create_intervals(preds=train_preds, data=train_data)

In [ ]:
valid_data = add_pred_stats(preds=valid_preds, data=valid_data)
valid_data = create_intervals(preds=valid_preds, data=valid_data)

In [ ]:
public_data = add_pred_stats(preds=public_preds, data=public_data)
public_data = create_intervals(preds=public_preds, data=public_data)

In [ ]:
private_data = add_pred_stats(preds=private_preds, data=private_data)
private_data = create_intervals(preds=private_preds, data=private_data)

In [ ]:
p_cols = [i for i in private_data.columns if i.startswith("p_")]

In [ ]:
for p in p_cols:
    data[p] = 0
    data[p].loc[data.day_id.isin(TRAIN_INDICES)] = train_data[p]
    data[p].loc[data.day_id.isin(VALID_INDICES)] = valid_data[p]
    data[p].loc[data.day_id.isin(PUBLIC_INDICES)] = public_data[p]
    data[p].loc[data.day_id.isin(PRIVATE_INDICES)] = private_data[p]

## Submission

In [ ]:
submission = data.loc[data['set'].isin(['Private', 'Public'])][['id',
                                                                'day_id',
                                                                'set'] + p_cols].copy().reset_index(drop=True)

In [ ]:
submission['day_id2'] = 'F' + pd.Series(np.where(submission['day_id'] <= max(PUBLIC_INDICES), 
                                                 submission['day_id'] - min(PUBLIC_INDICES) + 1, 
                                                 submission['day_id'] - min(PRIVATE_INDICES) + 1).astype(int).astype(str))

In [ ]:
submission['id'] = np.where(submission['set']=="Public",
                            submission['id'].str.replace("evaluation", "validation"), 
                            submission['id'])

In [ ]:
submission.groupby('set').agg({
    'day_id':[np.min, np.max]
})

In [ ]:
submission = submission.melt(id_vars=['id','day_id2','set'],
    value_vars=p_cols,
    var_name='alpha',
    value_name='prediction')

In [ ]:
submission['alpha'] = submission['alpha'].str.split("_").apply(lambda x: x[-1])

In [ ]:
def split_combine(x,y):
    return '_'.join(x.split("_")[:-1]+[y]+[x.split("_")[-1]])

In [ ]:
submission['id'] = submission.apply(lambda x: split_combine(x['id'], x['alpha']), axis=1)

In [ ]:
submission_file = submission.pivot(
    values='prediction',
    index='id',
    columns='day_id2').reset_index(drop=False)
submission_file = submission_file[['id']+["F"+str(i) for i in range(1,29,1)]]

In [ ]:
submission_file

In [ ]:
submission_file.head()

In [ ]:
submission_file.tail()

In [ ]:
submission_file.to_csv('../results/submission_'+MODEL_VERSION+'.csv', index=False)

In [ ]:
end_time = time()
end = datetime.now()
print(ctime(end_time))
print(end - start, model_t - start, model_t - end)